<a href="https://colab.research.google.com/github/nriverag/Modeling-of-atmospheric-particle-pollution-in-Bogota/blob/main/Redes_LTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math # Acá se importan las librerías a utilizar 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [3]:
from google.colab import files # Para cargar archivos en google colab 
uploaded = files.upload()

Saving PanelFiltrado.csv to PanelFiltrado.csv


In [112]:
import pandas as pd 
import io
df2= pd.read_csv("PanelFiltrado.csv", delimiter=",", error_bad_lines=False, header=0, decimal=",")
df2.index=pd.to_datetime(df2["Fecha"])

In [113]:
df2["Estación"].value_counts()

Fontibón                      54952
Puente Aranda                 54952
Bosa                          54952
Móvil Fontibón                54952
Kennedy                       54952
Suba                          54952
Las Ferias                    54952
Guaymaral                     54952
Centro de Alto Rendimiento    54952
Usme                          54952
Tunal                         54952
Ministerio de Ambiente        54952
San Cristóbal                 54952
Colina                        54952
Usaquén                       54952
Ciudad Bolívar                54952
Carvajal-Sevillana            54952
Bolivia                       54952
Name: Estación, dtype: int64

In [114]:
df2.head(10)

,Fecha,Fuente,Estación,EstaciónID,Año,Mes,Día,Hora,Unidad,PM10,PM2.5
Fecha,,,,,,,,,,,
2015-01-01 01:00:00,2015-01-01 01:00:00,RMCAB,Bolivia,1,2015,1,1,1,mg/m3,NaN,NaN
2015-01-01 02:00:00,2015-01-01 02:00:00,RMCAB,Bolivia,1,2015,1,1,2,mg/m3,NaN,NaN
2015-01-01 03:00:00,2015-01-01 03:00:00,RMCAB,Bolivia,1,2015,1,1,3,mg/m3,NaN,NaN
2015-01-01 04:00:00,2015-01-01 04:00:00,RMCAB,Bolivia,1,2015,1,1,4,mg/m3,NaN,NaN
2015-01-01 05:00:00,2015-01-01 05:00:00,RMCAB,Bolivia,1,2015,1,1,5,mg/m3,NaN,NaN
2015-01-01 06:00:00,2015-01-01 06:00:00,RMCAB,Bolivia,1,2015,1,1,6,mg/m3,NaN,NaN
2015-01-01 07:00:00,2015-01-01 07:00:00,RMCAB,Bolivia,1,2015,1,1,7,mg/m3,NaN,NaN
2015-01-01 08:00:00,2015-01-01 08:00:00,RMCAB,Bolivia,1,2015,1,1,8,mg/m3,NaN,NaN
2015-01-01 09:00:00,2015-01-01 09:00:00,RMCAB,Bolivia,1,2015,1,1,9,mg/m3,NaN,NaN


In [7]:
df2 = df2.astype({"PM2.5": 'float32'})
df2 = df2.astype({"PM10": 'float32'})
df2 = df2.astype({"Fecha": 'datetime64[ns]'})

In [115]:
# Tratamiento de fechas y columnas extras
guaymaral = df2[df2["Estación"] == "Guaymaral"]
guaymaral["Fecha"] = pd.date_range("01-01-2015 01:00:00", periods=54952, freq="H")
guaymaral.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Fecha,Fuente,Estación,EstaciónID,Año,Mes,Día,Hora,Unidad,PM10,PM2.5
Fecha,,,,,,,,,,,
2015-01-01 01:00:00,2015-01-01 01:00:00,RMCAB,Guaymaral,8,2015,1,1,1,mg/m3,40.0,19.2
2015-01-01 02:00:00,2015-01-01 02:00:00,RMCAB,Guaymaral,8,2015,1,1,2,mg/m3,51.0,41.0
2015-01-01 03:00:00,2015-01-01 03:00:00,RMCAB,Guaymaral,8,2015,1,1,3,mg/m3,153.0,118.3
2015-01-01 04:00:00,2015-01-01 04:00:00,RMCAB,Guaymaral,8,2015,1,1,4,mg/m3,120.0,94.3
2015-01-01 05:00:00,2015-01-01 05:00:00,RMCAB,Guaymaral,8,2015,1,1,5,mg/m3,57.0,40.2
2015-01-01 06:00:00,2015-01-01 06:00:00,RMCAB,Guaymaral,8,2015,1,1,6,mg/m3,53.0,37.0
2015-01-01 07:00:00,2015-01-01 07:00:00,RMCAB,Guaymaral,8,2015,1,1,7,mg/m3,61.0,41.2
2015-01-01 08:00:00,2015-01-01 08:00:00,RMCAB,Guaymaral,8,2015,1,1,8,mg/m3,43.0,33.7
2015-01-01 09:00:00,2015-01-01 09:00:00,RMCAB,Guaymaral,8,2015,1,1,9,mg/m3,51.0,18.7


In [116]:
# Split into training, validation and test datasets.
# Since it's timeseries we should do it by date.
test_cutoff_date = guaymaral['Fecha'].max() - timedelta(days=7)
val_cutoff_date = test_cutoff_date - timedelta(days=14)


df_test = guaymaral[guaymaral['Fecha'] > test_cutoff_date]
df_val = guaymaral[(guaymaral['Fecha'] > val_cutoff_date) & (guaymaral['Fecha'] <= test_cutoff_date)]
df_train = guaymaral[(guaymaral['Fecha'] <= val_cutoff_date)]
df_train.dropna(axis = 0)

#check out the datasets
print('Test dates: {} to {}'.format(df_test['Fecha'].min(), df_test['Fecha'].max()))
print('Validation dates: {} to {}'.format(df_val['Fecha'].min(), df_val['Fecha'].max()))
print('Train dates: {} to {}'.format(df_train['Fecha'].min(), df_train['Fecha'].max()))

Test dates: 2021-04-01 17:00:00 to 2021-04-08 16:00:00
Validation dates: 2021-03-18 17:00:00 to 2021-04-01 16:00:00
Train dates: 2015-01-01 01:00:00 to 2021-03-18 16:00:00


In [117]:
def create_ts_files(dataset, 
                    start_index, 
                    end_index, 
                    history_length, 
                    step_size, 
                    target_step, 
                    num_rows_per_file, 
                    data_folder):
    assert step_size > 0
    assert start_index >= 0
    
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    
    time_lags = sorted(range(target_step+1, target_step+history_length+1, step_size), reverse=True)
    col_names = [f'x_lag{i}' for i in time_lags] + ['y']
    start_index = start_index + history_length
    if end_index is None:
        end_index = len(dataset) - target_step
    
    rng = range(start_index, end_index)
    num_rows = len(rng)
    num_files = math.ceil(num_rows/num_rows_per_file)
    
    # for each file.
    print(f'Creating {num_files} files.')
    for i in range(num_files):
        filename = f'{data_folder}/ts_file{i}.pkl'
        
        if i % 10 == 0:
            print(f'{filename}')
            
        # get the start and end indices.
        ind0 = i*num_rows_per_file
        ind1 = min(ind0 + num_rows_per_file, end_index)
        data_list = []
        
        # j in the current timestep. Will need j-n to j-1 for the history. And j + target_step for the target.
        for j in range(ind0, ind1):
            indices = range(j-1, j-history_length-1, -step_size)
            data = dataset[sorted(indices) + [j+target_step]]
            
            # append data to the list.
            data_list.append(data)

        df_ts = pd.DataFrame(data=data_list, columns=col_names)
        df_ts.to_pickle(filename)
            
    return len(col_names)-1

In [118]:
%%time

global_active_power = df_train['PM10'].values

# Scaled to work with Neural networks.
scaler = MinMaxScaler(feature_range=(0, 1))
global_active_power_scaled = scaler.fit_transform(global_active_power.reshape(-1, 1)).reshape(-1, )

history_length = 7*24*60  # The history length in minutes.
step_size = 10  # The sampling rate of the history. Eg. If step_size = 1, then values from every minute will be in the history.
                #                                       If step size = 10 then values every 10 minutes will be in the history.
target_step = 10  # The time step in the future to predict. Eg. If target_step = 0, then predict the next timestep after the end of the history period.
                  #                                             If target_step = 10 then predict 10 timesteps the next timestep (11 minutes after the end of history).

# The csv creation returns the number of rows and number of features. We need these values below.
num_timesteps = create_ts_files(global_active_power_scaled,
                                start_index=0,
                                end_index=None,
                                history_length=history_length,
                                step_size=step_size,
                                target_step=target_step,
                                num_rows_per_file=128*100,
                                data_folder='ts_data')

Creating 4 files.
ts_data/ts_file0.pkl
CPU times: user 27.4 s, sys: 2.54 s, total: 29.9 s
Wall time: 29.6 s


In [127]:
class TimeSeriesLoader:
    def __init__(self, ts_folder, filename_format):
        self.ts_folder = ts_folder
        
        # find the number of files.
        i = 0
        file_found = True
        while file_found:
            filename = self.ts_folder + '/' + filename_format.format(i)
            file_found = os.path.exists(filename)
            if file_found:
                i += 1
                
        self.num_files = i
        self.files_indices = np.arange(self.num_files)
        self.shuffle_chunks()
        
    def num_chunks(self):
        return self.num_files
    
    def get_chunk(self, idx):
        assert (idx >= 0) and (idx < self.num_files)
        
        ind = self.files_indices[idx]
        filename = self.ts_folder + '/' + filename_format.format(ind)
        df_ts = pd.read_pickle(filename)
        num_records = len(df_ts.index)
        
        features = df_ts.drop('y', axis=1).values
        target = df_ts['y'].values
        
        # reshape for input into LSTM. Batch major format.
        features_batchmajor = np.array(features).reshape(num_records, -1, 1)
        return features_batchmajor, target
    
    # this shuffles the order the chunks will be outputted from get_chunk.
    def shuffle_chunks(self):
        np.random.shuffle(self.files_indices)

In [128]:
ts_folder = 'ts_data'
filename_format = 'ts_file{}.pkl'
tss = TimeSeriesLoader(ts_folder, filename_format)

In [129]:
ts_inputs = tf.keras.Input(shape=(num_timesteps, 1))

# units=10 -> The cell and hidden states will be of dimension 10.
#             The number of parameters that need to be trained = 4*units*(units+2)
x = layers.LSTM(units=10)(ts_inputs)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='linear')(x)
model = tf.keras.Model(inputs=ts_inputs, outputs=outputs)

In [130]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1008, 1)]         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                480       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [131]:
%%time

# train in batch sizes of 128.
BATCH_SIZE = 128
NUM_EPOCHS = 1
NUM_CHUNKS = tss.num_chunks()

for epoch in range(NUM_EPOCHS):
    print('epoch #{}'.format(epoch))
    for i in range(NUM_CHUNKS):
        X, y = tss.get_chunk(i)
        
        # model.fit does train the model incrementally. ie. Can call multiple times in batches.
        # https://github.com/keras-team/keras/issues/4446
        model.fit(x=X, y=y, batch_size=BATCH_SIZE)
        
    # shuffle the chunks so they're not in the same order next time around.
    tss.shuffle_chunks()

epoch #0
100/100 [==============================] - 22s 223ms/step - loss: nan - mse: nan
CPU times: user 2min 35s, sys: 6.83 s, total: 2min 42s
Wall time: 2min 8s


In [133]:
global_active_power_val = df_val['PM10'].values
global_active_power_val_scaled = scaler.transform(global_active_power_val.reshape(-1, 1)).reshape(-1, )

history_length = 7*24*60  # The history length in minutes.
step_size = 10  # The sampling rate of the history. Eg. If step_size = 1, then values from every minute will be in the history.
                #                                       If step size = 10 then values every 10 minutes will be in the history.
target_step = 10  # The time step in the future to predict. Eg. If target_step = 0, then predict the next timestep after the end of the history period.
                  #                                             If target_step = 10 then predict 10 timesteps the next timestep (11 minutes after the end of history).

# The csv creation returns the number of rows and number of features. We need these values below.
num_timesteps = create_ts_files(global_active_power_val_scaled,
                                start_index=0,
                                end_index=None,
                                history_length=history_length,
                                step_size=step_size,
                                target_step=target_step,
                                num_rows_per_file=128*100,
                                data_folder='ts_val_data')

Creating 0 files.


In [135]:
df_val_ts = pd.read_pickle('ts_val_data/ts_file0.pkl')

features = df_val_ts.drop('y', axis=1).values
features_arr = np.array(features)

# reshape for input into LSTM. Batch major format.
num_records = len(df_val_ts.index)
features_batchmajor = features_arr.reshape(num_records, -1, 1)


y_pred = model.predict(features_batchmajor).reshape(-1, )
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1 ,)

y_act = df_val_ts['y'].values
y_act = scaler.inverse_transform(y_act.reshape(-1, 1)).reshape(-1 ,)

print('validation mean squared error: {}'.format(mean_squared_error(y_act, y_pred)))

#baseline
y_pred_baseline = df_val_ts['x_lag11'].values
y_pred_baseline = scaler.inverse_transform(y_pred_baseline.reshape(-1, 1)).reshape(-1 ,)
print('validation baseline mean squared error: {}'.format(mean_squared_error(y_act, y_pred_baseline)))

FileNotFoundError: ignored